In [ ]:
import pandas as pd
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import logging
logging.basicConfig(level=logging.ERROR)
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/TagRecInference/data/JEE_questions.csv")

In [ ]:
data

,subject,question
0,Physics,Which of the following possesses Potential ene...
1,Physics,A car is being driven at a constant speed of \...
2,Physics,Which of the following statements is\\nincorre...
3,Physics,Point \\\\( A \\\\) is at a lower electrical p...
4,Chemistry,Assertion \\\\( (A): O_{3} \\\\) is used to\\n...
...,...,...
295,Physics,Distinguish between 'intrinsic' and\\nextrinsi...
296,Physics,When an electric current is passed through an ...
297,Physics,What measures are taken to reduce\\nfluid fric...
298,Chemistry,Which one of the following is incorrectly matc...


In [ ]:
data =data.dropna(subset=['question'])

In [ ]:
data[:6]

,subject,question
0,Physics,Which of the following possesses Potential ene...
1,Physics,A car is being driven at a constant speed of \...
2,Physics,Which of the following statements is\\nincorre...
3,Physics,Point \\\\( A \\\\) is at a lower electrical p...
4,Chemistry,Assertion \\\\( (A): O_{3} \\\\) is used to\\n...
5,Chemistry,What will be the ratio of molar\\ncapacities f...


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/TagRecInference/data/test_taxonomy_prediction.csv")
data = data[data['board_syllabus'].str.split('>>').str[0].str.strip().isin(['physics', 'chemistry'])]
data

,board_syllabus,question_answer
0,physics>>physics : part - ii,(a) Describe a simple experiment (or activity...
3,physics>>physics : part - i>>physical world,The branch of Physics that deals with the mic...
9,physics>>physics : part - i>>laws of motion,A curved road of radius 900 m is banked so th...
17,physics>>physics : part - i>>gravitation,A satellite is revolving around the earth at ...
18,chemistry>>chemistry : part i>>solutions,What are azeotropes? Give an example of maxim...
...,...,...
4767,chemistry>>chemistry : part i>>d and f- block ...,Number of electrons transferred in each case ...
4773,physics>>physics : part - i>>motion in a plane,Two objects are thrown with the same initial ...
4775,chemistry>>chemistry : part i>>the p-block ele...,The element showing maximum tendency for caten...
4777,chemistry>>chemistry : part i>>surface chemistry,A catalyst increases the rate of reaction beca...


In [ ]:
!pip install sentence-transformers==1.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.0-py3-none-any.whl size=114144 sha256=a029765ff22a2ee56499abbfd3ae12a374ec531b532bdec0711dce84b1b48219
  Stored in directory: /root/.cache/pip/wheels/4f/e7/fd/fc0bf76654a0cf224bd7577c54d9e58629f7ebf373c1cd8fa2
Successfully built sentence-transformers
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 5.1.2
    Uninstalling sentence-transformers-5.1.2:
      Successfully uninstalled sentence-transformers-5.1.2


In [ ]:
!pip install transformers==4.35.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 134.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 136.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1


In [ ]:
import re
def clean_sentence(question):
  # print(question)
  question = re.sub('<[^>]*>', ' ',question)
  question = re.sub(' +', ' ', question)
  question = re.sub('\xa0','',question)
  question = question.rstrip()
  question = re.sub('nan','',question)
  question = re.sub(u'\u2004','',question)
  question = re.sub(u'\u2009','',question)

  # question = question.decode("utf-8")
  # question = question.replace(u'\u200\d*','').encode("utf-8")
  question = re.sub('&nbsp','',question)
  question = re.sub('&ndash','',question)
  question = re.sub('\r','',question)
  question = re.sub('\t','',question)
  question = re.sub('\n',' ',question)

  question = re.sub('MathType@.*','',question)
  question = re.sub('&thinsp','',question)
  question = re.sub('&times','',question)
  question = re.sub('\u200b','',question)
  question = re.sub('&rarr;;;','',question)

  return question

In [ ]:
data["question"] = data["question_answer"].apply(lambda x: clean_sentence(x))
data

,board_syllabus,question_answer,question
0,physics>>physics : part - ii,(a) Describe a simple experiment (or activity...,(a) Describe a simple experiment (or activity...
3,physics>>physics : part - i>>physical world,The branch of Physics that deals with the mic...,The branch of Physics that deals with the mic...
9,physics>>physics : part - i>>laws of motion,A curved road of radius 900 m is banked so th...,A curved road of radius 900 m is banked so th...
17,physics>>physics : part - i>>gravitation,A satellite is revolving around the earth at ...,A satellite is revolving around the earth at ...
18,chemistry>>chemistry : part i>>solutions,What are azeotropes? Give an example of maxim...,What are azeotropes? Give an example of maxim...
...,...,...,...
4767,chemistry>>chemistry : part i>>d and f- block ...,Number of electrons transferred in each case ...,Number of electrons transferred in each case ...
4773,physics>>physics : part - i>>motion in a plane,Two objects are thrown with the same initial ...,Two objects are thrown with the same initial ...
4775,chemistry>>chemistry : part i>>the p-block ele...,The element showing maximum tendency for caten...,The element showing maximum tendency for caten...
4777,chemistry>>chemistry : part i>>surface chemistry,A catalyst increases the rate of reaction beca...,A catalyst increases the rate of reaction beca...


In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Loading BERT tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from sentence_transformers import SentenceTransformer
sent_model = SentenceTransformer('bert-large-nli-stsb-mean-tokens')

  0%|          | 0.00/1.24G [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:
import numpy as np
!pip install inflection

import inflection

from nltk.stem import PorterStemmer
ps = PorterStemmer()
from gzip import open as gopen
from pandas.core.common import flatten
def get_cleaned_taxonomy(taxonomy):
  cleaned_taxonomy = []
  for value in taxonomy:
      value = value.strip()
      value = ' '.join(value.split(">>"))
      # taxonomy_words = [inflection.singularize(val)  for token in value for val in token.split(" ") if val.isalpha()]
      cleaned_taxonomy.append( value )
  return cleaned_taxonomy

In [ ]:
with open("/content/drive/MyDrive/TagRecInference/data/ncert_taxonomy.txt", "r") as f:
    label_set = f.readlines()

In [ ]:
len(label_set)

357

In [ ]:
targets = get_cleaned_taxonomy(label_set)
len(targets)

357

In [ ]:
taxonomy_vectors = sent_model.encode(targets)
taxonomy_vectors = np.vstack(taxonomy_vectors)

taxonomy_vectors.shape

(357, 1024)

In [ ]:
train_poincare_tensor = torch.tensor(taxonomy_vectors,dtype=torch.float)

In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split


def get_tokenized_input(text):
    input_ids = []
    attention_masks = []

    for sent in text:
        # print("sent",sent)
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 128,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            truncation=True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )

        # Add the encoded sentence to the list.
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)


    # Print sentence 0, now as a list of IDs.
    print('Original: ', text[0])
    print('Token IDs:', input_ids[0])
    return input_ids, attention_masks

In [ ]:
input_ids, attention_masks = get_tokenized_input(data["question"].values)

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:   (a) Describe a simple experiment (or activity) to show that the polarity of emf induced in a coil is always such that it tends to produce a current which opposes the change of magnetic flux that produces it. (b) The current flowing through an inductor of self inductance L is continuously increasing. Plot a graph showing the variation of (i) Magnetic flux versus the current (ii) Induced emf versus dI/dt (iii) Magnetic potential energy stored versus the current. /* Style Definitions */ table.MsoNormalTable {mso-style-name:"Table Normal"; mso-tstyle-rowband-size:0; mso-tstyle-colband-size:0; mso-style-noshow:yes; mso-style-priority:99; mso-style-qformat:yes; mso-style-parent:""; mso-padding-alt:0in 5.4pt 0in 5.4pt; mso-para-margin-top:0in; mso-para-margin-right:0in; mso-para-margin-bottom:10.0pt; mso-para-margin-left:0in; line-height:115%; mso-pagination:widow-orphan; font-size:11.0pt; font-family:"Calibri","sans-serif"; mso-ascii-font-family:Calibri; mso-ascii-theme-font:mino

In [ ]:

import sys
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from matplotlib import pyplot as plt
from torch.nn.modules.loss import HingeEmbeddingLoss
from random import randint

from tqdm import tqdm
import time
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from matplotlib import pyplot as plt
from torch.nn.modules.loss import HingeEmbeddingLoss
from random import randint
import torch.nn.functional as F

import time
import argparse
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
cos_label = nn.CosineSimilarity(dim=1, eps=1e-5)

dist = torch.nn.PairwiseDistance(p=2.0, eps=1e-06)
nn.PairwiseDistance(p=2)
class MHSA(nn.Module):
  def __init__(self,
         emb_dim,
         kqv_dim,
         num_heads=2):
    super(MHSA, self).__init__()
    self.emb_dim = emb_dim
    self.kqv_dim = kqv_dim
    self.num_heads = num_heads

    self.w_k = nn.Linear(emb_dim, kqv_dim * num_heads, bias=False)
    self.w_q = nn.Linear(emb_dim, kqv_dim * num_heads, bias=False)
    self.w_v = nn.Linear(emb_dim, kqv_dim * num_heads, bias=False)
    self.w_out = nn.Linear(kqv_dim * num_heads, emb_dim)

  def forward(self, query, key, value):
    # print("query",query.shape)
    b, t = query.shape
    e = self.kqv_dim
    h = self.num_heads
    keys = self.w_k(key).view(b, h, e)
    values = self.w_v(value).view(b, h, e)
    queries = self.w_q(query).view(b, h, e)

    # keys = keys.transpose(2, 1)
    # queries = queries.transpose(2, 1)
    # values = values.transpose(2, 1)

    dot = queries @ keys.transpose(2, 1)  #(b*h*e) @ (b*e*h)
    dot = dot / np.sqrt(e)  # (b*h*h)
    dot = F.softmax(dot, dim=2)

    out = dot @ values   # (b*h*h) @ (b*h*e) = (b*h*e)
    out = out.contiguous().view(b, h * e)
    out = self.w_out(out)
    return out
# Neural Classifierwork

# Discussion TODOS
# try hierarhical interaction (TODO)

# try bringing in modalities (image, or video)

# Go from classical algorithm -> deep learning

class MulticlassClassifier(nn.Module):
    def __init__(self,bert_model_path):
        super(MulticlassClassifier,self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_path,output_hidden_states=True,output_attentions=False)
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(768, 1024)
        self.fc2 = nn.Linear(576, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.act = torch.nn.ReLU()
        self.fc4 = nn.Linear(512, 1024)

        self.multi_head_attention = MHSA(1024, 64,16)
        self.multihead_attn = torch.nn.MultiheadAttention(embed_dim = 1024,  num_heads = 4, batch_first=True)


    def forward(self,tokens,masks, targets=None, skip_attention=False):
        # print("tokens", tokens.shape)
        outputs = self.bert(tokens, attention_mask=masks)[2]
        # outputs[2] = outputs[2].permute(0,2,1)
        # print(outputs[-2].shape)
        output_1 = outputs[-1].permute(1,0,2)
        # print(outputs[1].shape,outputs[0].shape)
        output_1 = torch.mean(output_1, dim=0)
        # output_2 = outputs[-2].permute(1,0,2)
        # output_2 = torch.mean(output_2, dim=0)
        # print("output_2", output_2.shape, output_1.shape)
        pooled_output = outputs[-1] # output_1 # torch.cat((output_1, output_2), dim=1)
        # print("pooled_output", pooled_output.shape)
        x = self.fc1(pooled_output)
        # x = self.fc2(x)

        targets_curr_batch = []
        for index_1, input_x in enumerate(x):
            # print(input_x.shape, torch.mean(input_x,dim=0).shape)
            distance = cos_label(torch.mean(input_x,dim=0).reshape(1,-1), unique_poincare_tensor)
            distances,indices = torch.topk(distance,1,largest=True)

            target_distances = (F.normalize(unique_poincare_tensor[indices],p=2,dim=1) - F.normalize(unique_poincare_tensor,p=2,dim=1)).pow(2).sum(1) #cos_label(unique_poincare_tensor[indices].reshape(1,-1), unique_poincare_tensor)
            distances,indices = torch.topk(target_distances,1,largest=False)
            targets_curr_batch.append(unique_poincare_tensor[indices].reshape(1,1,1024))
            # print("here")
        # print(len(targets_curr_batch))
        targets_batch = torch.cat(targets_curr_batch, dim=0)
        # print("targets_batch",targets_batch.shape)
        # if not skip_attention:
          # print("here attention")
        attn_output, attn_output_weights = self.multihead_attn(targets_batch, x, x)
        # target_attn_output, attn_output_weights = self.multihead_attn(x, targets_batch, targets_batch)

        x = torch.sum(attn_output,dim=1)
        # target_final = torch.mean(target_attn_output, dim=1)
        # x = self.act(x)
        # x = self.fc4(x)
        # print("X shape",targets_batch.unsqueeze(dim=1).shape,x.shape,attn_output.shape)
        return x

class MyHingeLoss(torch.nn.Module):
    def __init__(self, margin):
        super(MyHingeLoss, self).__init__()
        self.margin = margin
    # def forward_val(self, output, target):
    #     cos = nn.CosineSimilarity(dim=0, eps=1e-6)
    #     loss = 0
    #     num_compare = 4
    #     count = 0
    #     for i in range(len(output)):
    #         v_image = output[i]
    #         t_label = target[i]
    #         for j in range(num_compare):
    #             if j != i:
    #                 count += 1
    #                 t_j = target[j]
    #                 loss += torch.relu( self.margin - cos(t_label, v_image) + cos(t_j, v_image) )
    #     return loss / count

    def forward(self, output, target):
        loss=0
        for i in range(len(output)):
            v_image = F.normalize(output[i],p=2,dim=0)
            t_label = F.normalize(target[i],p=2,dim=0)
            # j = randint(0, len(output)-1)
            # while j == i:
            #     j = randint(0, len(output)-1)
            distance = cos_label(t_label, target)
            # print(distance.shape)
            delta = min(len(target)-2,8)
            distances,indices = torch.topk(distance,len(target)-delta,largest=True)
            # print(indices)
            # index_target = random.choice(indices)
            # while index_target == i:
            #     # print("here***", index_target,i)
            #     index_target = random.choice(indices)
            count = 0
            for index_target in indices:
                if index_target!=i:
                    count=count+1
                    t_j = F.normalize(target[index_target],p=2,dim=0)
                    loss+= torch.relu( self.margin - cos(t_label, v_image) + cos(t_j, v_image) )
            # print("count",count)
        return loss / (len(output) * count)


In [ ]:
from transformers import BertModel, AdamW, BertConfig
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertModel, AdamW, BertConfig

# Loads BertModel, the pretrained BERT model with a single
model = MulticlassClassifier('bert-base-uncased')
model.load_state_dict(torch.load('/content/drive/MyDrive/TagRecInference/model_weights.zip'))

# Tell pytorch to run this model on the GPU.
model.cuda()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

MulticlassClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [ ]:
def get_inference_taxonomies(input_ids, attention_masks, label_set, train_poincare_tensor):
    model.eval()
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

    input_ids = input_ids.to('cuda')
    attention_masks = attention_masks.to('cuda')
    train_poincare_tensor = train_poincare_tensor.to('cuda')
    # Tracking variables
    predictions , true_labels = [], []
    for input_id,attention_mask in zip(input_ids, attention_masks):
        with torch.no_grad():
            outputs = model(input_id.reshape(1,-1),attention_mask.reshape(1,-1))

        distances = cos(outputs,train_poincare_tensor)
        distances,indices = torch.topk(distances,3,largest=True)
        label_set = np.array(label_set)
        predictions.append(label_set[indices.cpu().numpy()])
        print(len(predictions))
    return predictions


In [ ]:
unique_poincare_tensor = train_poincare_tensor.to(device)
predictions = get_inference_taxonomies(input_ids, attention_masks, label_set, train_poincare_tensor)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
predictions

[array(['\n', '\n',
        'Physics >> Units and Measurements >> The international system of units\n'],
       dtype='<U144'),
 array(['Physics >> Units and Measurements >> Dimensions of physical quantities\n',
        'Physics >> Dual Nature of Radiation and Matter >> Particle Nature of Light: The Photon\n',
        'Physics >> Dual Nature of Radiation and Matter >> Photoelectric Effect\n'],
       dtype='<U144'),
 array(['Physics >> System of Particles and Rotational Motion >> Linear momentum of a system of particles\n',
        'Physics >> System of Particles and Rotational Motion >> Angular velocity and its relation with linear velocity\n',
        'Physics >> System of Particles and Rotational Motion >> Angular momentum in case of rotations about a fixed axis\n'],
       dtype='<U144'),
 array(['Physics >> System of Particles and Rotational Motion >> Centre of mass\n',
        'Physics >> System of Particles and Rotational Motion >> Motion of centre of mass\n',
        'Physics >

In [ ]:
final_predictions = []
for pred in predictions:
    pred_list = pred.tolist()
    predictions_1 = (',').join(pred_list)
    final_predictions.append(predictions_1)
final_predictions = pd.Series(final_predictions, dtype = str)
data["predictions"] = final_predictions
data

,board_syllabus,question_answer,question,predictions
0,physics>>physics : part - ii,(a) Describe a simple experiment (or activity...,(a) Describe a simple experiment (or activity...,"\n,\n,Physics >> Units and Measurements >> The..."
3,physics>>physics : part - i>>physical world,The branch of Physics that deals with the mic...,The branch of Physics that deals with the mic...,Physics >> System of Particles and Rotational ...
9,physics>>physics : part - i>>laws of motion,A curved road of radius 900 m is banked so th...,A curved road of radius 900 m is banked so th...,Physics >> Semiconductor Electronics: Material...
17,physics>>physics : part - i>>gravitation,A satellite is revolving around the earth at ...,A satellite is revolving around the earth at ...,"Chemistry >> Solutions >> Solubility\n,Chemist..."
18,chemistry>>chemistry : part i>>solutions,What are azeotropes? Give an example of maxim...,What are azeotropes? Give an example of maxim...,Physics >> Electromagnetic Waves >> Electromag...
...,...,...,...,...
4767,chemistry>>chemistry : part i>>d and f- block ...,Number of electrons transferred in each case ...,Number of electrons transferred in each case ...,NaN
4773,physics>>physics : part - i>>motion in a plane,Two objects are thrown with the same initial ...,Two objects are thrown with the same initial ...,NaN
4775,chemistry>>chemistry : part i>>the p-block ele...,The element showing maximum tendency for caten...,The element showing maximum tendency for caten...,NaN
4777,chemistry>>chemistry : part i>>surface chemistry,A catalyst increases the rate of reaction beca...,A catalyst increases the rate of reaction beca...,NaN


In [ ]:
data.to_csv("prediction.csv", index=False)

In [ ]:
!cp prediction.csv  "/content/drive/MyDrive/TagRecInference"